Slice and Dice

In [ ]:
from datasets import load_dataset

In [ ]:
url='https://github.com/crux82/squad-it/raw/master/'
data={
    'train':url+"SQuAD_it-train.json.gz",
    'test':url+"SQuAD_it-test.json.gz"
}
sq_dataset=load_dataset('json',data_files=data,field='data')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
!wget 'https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip'

--2025-09-23 15:53:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/00462/drugsCom_raw.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘drugsCom_raw.zip’

drugsCom_raw.zip        [       <=>          ]  41.00M  33.1MB/s    in 1.2s    

2025-09-23 15:53:58 (33.1 MB/s) - ‘drugsCom_raw.zip’ saved [42989872]



In [ ]:
!unzip drugsCom_raw.zip

Archive:  drugsCom_raw.zip
  inflating: drugsComTest_raw.tsv    
  inflating: drugsComTrain_raw.tsv   


In [ ]:
data={'train':'drugsComTrain_raw.tsv','test':'drugsComTest_raw.tsv'}
drug_data=load_dataset('csv',data_files=data,delimiter='\t')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
drug_data['train'].shuffle(seed=132).select(range(1000))[:3]

{'Unnamed: 0': [196606, 226367, 58138],
 'drugName': ['Letrozole', 'Etonogestrel', 'Acetaminophen / hydrocodone'],
 'condition': ['Breast Cance', 'Birth Control', None],
 'review': ['"I&#039;m 61 years of age. Approaching my 4 th year  of taking letrozole Accord. I am suffering with joint ache which attacks my knees,one ankle. Wrists, elbows but has now affected my hands, most percularilly my middle fingers of each hand! I also get pins and needles in my hands. The morning stiffness is throughout my joints and takes a while to get going. I have calf pain and muscle weakness and blurry eyesight too sometimes. I don&#039;t complain as its prevented the cancer returning.I do dispare at my poor quality of life. I take calcium, magnesium and pain relief.I  walk every day and exercise . I&#039;ve also gained weight.   I&#039;m glad to know there are others with similar side effects but even more glad that I&#039;m here to share my story ."',
  '"Nexplanon was a mistake for me. It caused me n

In [ ]:
for split in drug_data.keys():
  assert len(drug_data[split])==len(drug_data[split].unique('Unnamed: 0'))

In [ ]:
drug_data=drug_data.rename_column('Unnamed: 0','patient_id')

In [ ]:
drug_data

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 161297
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount'],
        num_rows: 53766
    })
})

In [ ]:
len(drug_data['train'].unique('drugName'))

3436

In [ ]:
drug_data=drug_data.filter(lambda x:x['condition'] is not None)

Filter:   0%|          | 0/161297 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53766 [00:00<?, ? examples/s]

In [ ]:
def lower_case(eg):
  return {'condition':eg['condition'].lower()}

In [ ]:
drug_data=drug_data.map(lower_case)

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [ ]:
drug_data['train']['condition'][5:10]

['benign prostatic hyperplasia',
 'emergency contraception',
 'bipolar disorde',
 'epilepsy',
 'birth control']

In [ ]:
def compute_length(eg):
  return{'review_length':len(eg['review'].split())}

In [ ]:
drug_data=drug_data.map(compute_length)

Map:   0%|          | 0/160398 [00:00<?, ? examples/s]

Map:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [ ]:
drug_data['train'][4]

{'patient_id': 35696,
 'drugName': 'Buprenorphine / naloxone',
 'condition': 'opiate dependence',
 'review': '"Suboxone has completely turned my life around.  I feel healthier, I&#039;m excelling at my job and I always have money in my pocket and my savings account.  I had none of those before Suboxone and spent years abusing oxycontin.  My paycheck was already spent by the time I got it and I started resorting to scheming and stealing to fund my addiction.  All that is history.  If you&#039;re ready to stop, there&#039;s a good chance that suboxone will put you on the path of great life again.  I have found the side-effects to be minimal compared to oxycontin.  I&#039;m actually sleeping better.   Slight constipation is about it for me.  It truly is amazing. The cost pales in comparison to what I spent on oxycontin."',
 'rating': 9.0,
 'date': 'November 27, 2016',
 'usefulCount': 37,
 'review_length': 124}

In [ ]:
drug_data['train'].sort('review_length')[:5]

{'patient_id': [111469, 13653, 53602, 135265, 160223],
 'drugName': ['Ledipasvir / sofosbuvir',
  'Amphetamine / dextroamphetamine',
  'Alesse',
  'Zegerid',
  'Rivaroxaban'],
 'condition': ['hepatitis c',
  'adhd',
  'birth control',
  'gerd',
  'deep vein thrombosis'],
 'review': ['"Headache"', '"Great"', '"Awesome"', '"expensive"', '"Good"'],
 'rating': [10.0, 10.0, 10.0, 10.0, 9.0],
 'date': ['February 3, 2015',
  'October 20, 2009',
  'November 23, 2015',
  'March 12, 2008',
  'December 8, 2013'],
 'usefulCount': [41, 3, 0, 32, 11],
 'review_length': [1, 1, 1, 1, 1]}

In [ ]:
drug_data=drug_data.filter(lambda x:x['review_length']>30)

Filter:   0%|          | 0/160398 [00:00<?, ? examples/s]

Filter:   0%|          | 0/53471 [00:00<?, ? examples/s]

In [ ]:
import html

In [ ]:
drug_data=drug_data.map(lambda x:{'review':html.unescape(x['review'])})

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def tokenizer_split(eg):
  return tokenizer(
      eg['review'],
      truncation=True,
      max_length=128,
      return_overflowing_tokens=True
  )

In [ ]:
tokenized_data=drug_data.map(tokenizer_split,batched=True,remove_columns=drug_data['train'].column_names)

Map:   0%|          | 0/138514 [00:00<?, ? examples/s]

Map:   0%|          | 0/46108 [00:00<?, ? examples/s]

In [ ]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 204198
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'overflow_to_sample_mapping'],
        num_rows: 68023
    })
})

In [ ]:
drug_dataset_clean=drug_data['train'].train_test_split(train_size=0.8,seed=123)
drug_dataset_clean['validation']=drug_dataset_clean.pop('test')
drug_dataset_clean['test']=drug_data['test']

In [ ]:
drug_dataset_clean

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 110811
    })
    validation: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 27703
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

In [ ]:
drug_dataset_clean.save_to_disk('drug-review')

Saving the dataset (0/1 shards):   0%|          | 0/110811 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/27703 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/46108 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk

In [ ]:
drug_data_loaded=load_from_disk('drug-review')

In [ ]:
drug_data

DatasetDict({
    train: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 138514
    })
    test: Dataset({
        features: ['patient_id', 'drugName', 'condition', 'review', 'rating', 'date', 'usefulCount', 'review_length'],
        num_rows: 46108
    })
})

Big Data

In [ ]:
pip install zstandard

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("casinca/PUBMED_title_abstracts_2019_baseline")

README.md: 0.00B [00:00, ?B/s]

PUBMED_title_abstracts_2019_baseline.jso(…):   0%|          | 0.00/6.86G [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/42 [00:00<?, ?it/s]

In [ ]:
dataset['train'][5]

{'meta': {'pmid': 11409579, 'language': 'eng'},
 'text': 'Oxygen delivery to children with hypoxaemia in small hospitals in developing countries.\nOxygen administration is one of the most important modalities of therapy for a patient with hypoxaemia to prevent death. This review summarises the methods of oxygen delivery applicable in small hospitals in developing countries, and evaluates information about their safety and efficacy. The following criteria were considered: cost and availability, efficiency/oxygen concentration achieved, tolerability/comfort of the method, requirement of humidification, demand for nursing care, and safety of the method and complications. In summary, it is concluded that all low-flow methods, i.e., nasopharyngeal catheters, nasal catheters and prongs, are effective in the oxygenation of sick children with severe pneumonia or bronchiolitis. Nasal prongs are the safest method of oxygen delivery, but nasopharyngeal catheters and nasal catheters are more easil

In [ ]:
pip install psutil

In [ ]:
import psutil
psutil.Process().memory_info().rss/(1024*1024)

1785.96875

In [ ]:
dataset1 = load_dataset("casinca/PUBMED_title_abstracts_2019_baseline",split='train',streaming=True)

In [ ]:
next(iter(dataset1))

{'meta': {'pmid': 11409574, 'language': 'eng'},
 'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seventeen published studies were found that i

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('distilbert-base-uncased')
tdata=dataset1.map(lambda x:tokenizer(x['text']))

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
next(iter(tdata))

{'meta': {'pmid': 11409574, 'language': 'eng'},
 'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seventeen published studies were found that i

In [ ]:
sdata=dataset1.shuffle(buffer_size=10000,seed=98)
next(iter(sdata))

{'meta': {'pmid': 11414525, 'language': 'eng'},
 'text': 'Spontaneous rupture of the liver presenting as scrotal bruising.\nWe describe a case of spontaneous rupture of the liver in a neonate, presenting with features consistent with testicular torsion. Ruptured liver has been well described in neonates following birth trauma, coagulation defects, and Group B Streptococcal infection. The etiology in our case remains undefined.'}

In [ ]:
list(dataset1.take(3))

[{'meta': {'pmid': 11409574, 'language': 'eng'},
  'text': 'Epidemiology of hypoxaemia in children with acute lower respiratory infection.\nTo determine the prevalence of hypoxaemia in children aged under 5 years suffering acute lower respiratory infections (ALRI), the risk factors for hypoxaemia in children under 5 years of age with ALRI, and the association of hypoxaemia with an increased risk of dying in children of the same age. Systematic review of the published literature. Out-patient clinics, emergency departments and hospitalisation wards in 23 health centres from 10 countries. Cohort studies reporting the frequency of hypoxaemia in children under 5 years of age with ALRI, and the association between hypoxaemia and the risk of dying. Prevalence of hypoxaemia measured in children with ARI and relative risks for the association between the severity of illness and the frequency of hypoxaemia, and between hypoxaemia and the risk of dying. Seventeen published studies were found that

In [ ]:
train=sdata.skip(1000)
validation=sdata.take(1000)

Creating own dataset

In [ ]:
import requests

In [ ]:
url='https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1'
response=requests.get(url)

In [ ]:
response.status_code

200

In [ ]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/7785',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/7785/events',
  'html_url': 'https://github.com/huggingface/datasets/pull/7785',
  'id': 3439897018,
  'node_id': 'PR_kwDODunzps6pyTM_',
  'number': 7785,
  'title': 'Fix Audio docstring by removing unsupported mono argument',
  'user': {'login': 'tanuj-rai',
   'id': 84439872,
   'node_id': 'MDQ6VXNlcjg0NDM5ODcy',
   'avatar_url': 'https://avatars.githubusercontent.com/u/84439872?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/tanuj-rai',
   'html_url': 'https://github.com/tanuj-rai',
   'followers_url': 'https://api.github.com/users/tanuj-rai/followers',
   'follo

Semantic search with FAISS

In [ ]:
issues=load_dataset('lewtun/github-issues',split='train')

README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


datasets-issues-with-comments.jsonl:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3019 [00:00<?, ? examples/s]

In [ ]:
issues

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 3019
})

In [ ]:
issues=issues.filter(lambda x:(x['is_pull_request']==False and len(x['comments'])>0))

Filter:   0%|          | 0/3019 [00:00<?, ? examples/s]

In [ ]:
issues

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'pull_request', 'body', 'timeline_url', 'performed_via_github_app', 'is_pull_request'],
    num_rows: 808
})

In [ ]:
columns=issues.column_names
keep=['title','body','html_url','comments']
remove=set(keep).symmetric_difference(columns)
issues=issues.remove_columns(remove)

In [ ]:
issues

Dataset({
    features: ['html_url', 'title', 'comments', 'body'],
    num_rows: 808
})

In [ ]:
issues.set_format('pandas')
df=issues[:]

In [ ]:
df['comments'][5]

array(['Why 1.9 ?\r\n\r\nhttps://anaconda.org/HuggingFace/datasets currently says 1.11',
       'Alright I added 1.12.0 and 1.12.1 and fixed the conda build #2952 '],
      dtype=object)

In [ ]:
comments_df=df.explode('comments',ignore_index=True)

In [ ]:
comments_df.head(5)

,html_url,title,comments,body
0,https://github.com/huggingface/datasets/issues...,Protect master branch,"Cool, I think we can do both :)",After accidental merge commit (91c55355b634d0d...
1,https://github.com/huggingface/datasets/issues...,Protect master branch,@lhoestq now the 2 are implemented.\r\n\r\nPle...,After accidental merge commit (91c55355b634d0d...
2,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Hi ! I guess the caching mechanism should have...,## Describe the bug\r\nAfter upgrading to data...
3,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,"If it's easy enough to implement, then yes ple...",## Describe the bug\r\nAfter upgrading to data...
4,https://github.com/huggingface/datasets/issues...,Backwards compatibility broken for cached data...,Well it can cause issue with anyone that updat...,## Describe the bug\r\nAfter upgrading to data...


In [ ]:
from datasets import Dataset

In [ ]:
comments_data=Dataset.from_pandas(comments_df)

In [ ]:
comments_data=comments_data.map(lambda x:{'comment_length':len(x['comments'].split())})

Map:   0%|          | 0/2964 [00:00<?, ? examples/s]

In [ ]:
comments_data=comments_data.filter(lambda x:x['comment_length']>15)

Filter:   0%|          | 0/2964 [00:00<?, ? examples/s]

In [ ]:
def concate(eg):
  return{
      'text':eg['title']+'\n'+eg['body']+'\n'+eg['comments']
  }

In [ ]:
comments_data=comments_data.map(concate)

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

In [ ]:
comments_data

Dataset({
    features: ['html_url', 'title', 'comments', 'body', 'comment_length', 'text'],
    num_rows: 2175
})

In [ ]:
from transformers import AutoModel,AutoTokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model=AutoModel.from_pretrained('sentence-transformers/multi-qa-mpnet-base-dot-v1')

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
def cls(output):
  return output.last_hidden_state[:,0]

In [ ]:
def embedding(text):
  encoded_input=tokenizer(text,padding=True,truncation=True,return_tensors='pt')
  encoded_input={k:v for k,v in encoded_input.items()}
  model_output=model(**encoded_input)
  return cls(model_output)

In [ ]:
embedding(comments_data['text'][0])

tensor([[-1.5532e-01, -1.0023e-01, -7.0321e-02, -7.9816e-02, -1.0425e-01,
         -1.8799e-01,  1.0403e-02,  2.7286e-01, -9.8491e-03, -8.4140e-02,
          2.9261e-01, -7.7875e-02, -1.3246e-01,  2.1589e-01, -5.6229e-02,
          1.7055e-01,  2.0032e-01, -4.0798e-02, -9.2333e-02,  2.7923e-02,
          2.2609e-02, -6.8703e-02,  8.3200e-02,  4.3631e-02, -1.8378e-01,
          5.4009e-03, -1.4812e-02,  1.5650e-01, -4.3977e-01, -4.6957e-01,
          1.4525e-01,  2.3224e-01,  2.5106e-02,  5.7053e-01, -1.0436e-04,
         -3.4572e-04,  1.6949e-01, -4.0601e-02, -1.4010e-01, -2.1546e-01,
         -4.9011e-01, -4.4480e-01, -7.0063e-02, -8.3547e-03,  1.2881e-01,
          5.2868e-02, -1.3250e-01,  2.3845e-02,  3.8485e-01,  1.0555e-01,
          2.3861e-01, -1.0285e-01,  1.8124e-01,  1.3770e-02,  3.9201e-01,
          5.4745e-01, -6.6635e-02,  3.0320e-01,  5.4018e-02,  6.7669e-04,
          6.1106e-02,  1.7411e-01, -8.2372e-02, -2.8667e-01,  2.2702e-01,
         -1.7799e-01,  5.1817e-01, -4.

In [ ]:
embedding_data=comments_data.map(lambda x:{'embeddings':embedding(x['text']).detach().numpy()[0]})

Map:   0%|          | 0/2175 [00:00<?, ? examples/s]

In [ ]:
embedding_data.add_faiss_index(column='embeddings')

In [ ]:
quest='How can I load a dataset offline?'
quest_embed=embedding([quest]).detach().numpy()

In [ ]:
scores_samples=embedding_data.get_nearest_examples('embeddings',quest_embed,k=7)

In [ ]:
samples_df=pd.DataFrame.from_dict(samples)
samples_df['Scores']=scores
samples_df.sort_values('scores',ascending=False,inplace=True)

In [ ]:
for _,row in samples_df.iterrows():
  print(f"COMMENT: {row.comments}")
  print(f"SCORE: {row.scores}")
  print(f"TITLE: {row.title}")
  print(f"URL: {row.html_url}")
  print("=" * 50)
  print()